In [1]:
import os

In [2]:
%pwd


'c:\\Users\\ben jazia\\OneDrive\\Documents\\GitHub\\Chicken-Disease-MLOps project\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from cnnChickenDiseaseClassifier.constants import *
from cnnChickenDiseaseClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager: # we'll just add a get_prepare_callback_config method to the old Config Manager
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [ ]:
import os
import tensorflow as tf
import time

In [11]:
# the component
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig): # takes a config object that tells it where to save logs and checkpoints
        self.config = config

    
    @property
    def _create_tb_callbacks(self): # Builds a TensorBoard callback.
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S") 
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}", # timestamp ensures each training run gets a unique folder (avoids overwriting old logs).
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True # Saves model weights only when validation metrics improve
        ) # Ensures you always end training with the best model saved.
 

    def get_tb_ckpt_callbacks(self): # returns both callbacks as a list
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2025-09-11 18:31:34,469: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-11 18:31:34,472: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-11 18:31:34,473: INFO: common: created directory at: artifacts]
[2025-09-11 18:31:34,475: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-09-11 18:31:34,477: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [10]:

import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2025-09-11-17-41-51'